### 1. Importar Librerías:

In [1]:
import pandas as pd
import warnings
import re

warnings.filterwarnings('ignore')

### 2. Abrir Archivo:

In [2]:
# dataset descargado de https://climate.cityofnewyork.us/initiatives/nyc-greenhouse-gas-inventories/
URL= "https://drive.google.com/uc?id=1ZFhHtf64BgVZhzneOv1WhsyXXIX8ZyPn&export=download"
df = pd.read_excel(URL)

"Esto lo sube streamlit"

'Esto lo sube streamlit'

### 3. Normalización de tabla (desnaturalización):

In [3]:
# Seleccionamos las columnas que vamos a utilizar
columns_to_keep = ['Sectors Sector', 'Category Full', 'Category Label','Source Label','Source Units'] + [col for col in df.columns if col.startswith('CY')]

# Crear un nuevo DataFrame con las columnas seleccionadas
df_normalized = df[columns_to_keep].copy()

# Realizar la operación de fundido (melt)
df_normalized = pd.melt(df_normalized, id_vars=['Sectors Sector', 'Category Full', 'Category Label','Source Label','Source Units'], var_name='Año', value_name='Valor')

# Seleccionamos las columnas que vamos a utilizar
columns_to_keep = ['Sectors Sector', 'Category Full', 'Category Label','Source Label','Source Units'] + [col for col in df.columns if col.startswith('CY')]

# Crear un nuevo DataFrame con las columnas seleccionadas
df_normalized = df[columns_to_keep].copy()

# Realizar la operación de fundido (melt)
df_normalized = pd.melt(df_normalized, id_vars=['Sectors Sector', 'Category Full', 'Category Label','Source Label','Source Units'], var_name='Año', value_name='Valor')

# Crear una columna auxiliar 'Tipo' y procesar la columna 'Año'
df_normalized['Tipo'] = df_normalized['Año'].apply(lambda x: re.findall(r'(?:\d{4})\s(.+)', x)[0])
df_normalized['Año'] = df_normalized['Año'].apply(lambda x: re.findall(r'(\d{4})', x)[0])


### 4. Normalizar columna Tipo:

In [4]:
# Filtrar solo las filas donde el tipo es 'Consumed' o 'tCO2e'
df_normalized = df_normalized[df_normalized['Tipo'].isin(['Consumed', 'tCO2e','Source MMBtu'])]

# Utilizar pivot_table para convertir las categorías de 'Tipo' en columnas
df_pivot = df_normalized.pivot_table(index=['Año','Sectors Sector', 'Category Full', 'Category Label','Source Label','Source Units'],
                                     columns='Tipo', values='Valor', aggfunc='sum').reset_index()


### 5. Eliminar Nulos y duplicados (de existir):

In [5]:
# Eliminar registros nulos
df_pivot.dropna(inplace=True)

# Eliminar registros duplicados
df_pivot.drop_duplicates(inplace=True)

In [6]:
df_pivot

Tipo,Año,Sectors Sector,Category Full,Category Label,Source Label,Source Units,Consumed,Source MMBtu,tCO2e
0,2005,Stationary Energy,"Commercial and Institutional (commercial, inst...",Commercial and Institutional,#2 fuel oil,gallon,8.309925e+07,11582374.12,859744.89
1,2005,Stationary Energy,"Commercial and Institutional (commercial, inst...",Commercial and Institutional,#4 fuel oil,gallon,1.389649e+07,2049176.81,152808.20
2,2005,Stationary Energy,"Commercial and Institutional (commercial, inst...",Commercial and Institutional,#6 fuel oil,gallon,2.378685e+07,3603707.85,268944.84
3,2005,Stationary Energy,"Commercial and Institutional (commercial, inst...",Commercial and Institutional,Biofuel,gallon,1.811172e+05,23414.83,1.19
4,2005,Stationary Energy,"Commercial and Institutional (commercial, inst...",Commercial and Institutional,Downstream Natural gas,SCF,5.512830e+10,59389720.15,0.00
...,...,...,...,...,...,...,...,...,...
913,2022,Transportation,Railways,Railways,Upstream Electricity,kWh,2.133596e+09,18563567.77,0.00
914,2022,Waste,Biological treatment,Biological treatment,CH4 and N2O from composting,MT,4.524000e+04,0.00,8663.46
915,2022,Waste,Landfills,Landfills,CH4,MT,6.266992e+06,0.00,1912703.71
916,2022,Waste,Wastewater treatment,Wastewater treatment,CH4,MT,3.192620e+03,0.00,89393.30


In [61]:
#df_pivot.to_csv('../DataClean/CO2_NYC.csv',index=False)